# Immigration Data ETL
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

##### Introduction

This notebook contains my comments on Step 5 of the Project Template. Here a short intro to the project:

    Purpose of the project is to create an analytics dataset on US travellers that helps **FlyBy Salad**, a fast food franchise, to identify customer groups based on demographics, airport locations, travel status and dates.
    
    The project analyses the data using Pandas and Pyspark (Step 1 and 2). Then a Data Model is defined (Step 3).
    
    Then in Step 4 the project implements the data model in an ETL script that transforms immmigration data, airport and demographics information and saves this as Parquet files to the desired location.
    
    The data *could* also be loaded into a Redshift instance. The Parquet format includes the schema information for Redshift, so using the COPY command should work quite well. However this has not been implemented and tested and been taken out of scope.
    

#### Rationale

The technology of choice here is Apache Spark because of it's scalability in processing large sets of data.

While Airport and Demographic data could be handled even by Microsoft Excel the immigration data for April 2016 contains over 3 Million lines of data. This amount is too much and other months could contain even more data.

In Step 1 I explored the data also using Pandas. But this only worked for a sample of the data. I wrote an explicit function to read only x-thousand lines from the SAS file in order to keep good runtime performance.

Spark proved also a valid tool for assessing the data since PySpark implements basically a lot of Pandas' capabilities.

For production use the ETL process needs to be scheduled and monitored. I recommend here Apache Airflow. Using other scheduling tools could be possible but this is beyond my knowledge.

I developed all code in Jupyter notebooks as an "IDE". I tried to keep the code as simply and "pluggable" as possible. Then I copied everything into the "etl.py" and "nb_helpers.py" for commandline execution.

##### Spark scalability and performance improvements
* The ETL script takes about 15 minutes to complete
* Working with a sample of 1% results in a smaller immigration fact table, but the script's runtime does not significantly change
* This means that a **100-times increase in data** had **no significant impact** on processing
* Actually my test results were:

| Sample size | QA Checks | Output to S3 | Runtime         |
|-------------|-----------|--------------|-----------------|
| 100% | YES | NO | 20 minutes |
| 100% | NO | NO |  6:30 minutes |
| 1% | YES | NO |  16 minutes |
| 1% | NO | NO |  8 minutes |
| 100% | NO | YES |  6:20 minutes |
| 1% | NO | YES |  5:41 minutes |
| 1% | YES | YES | 16:27 minutes


#### Scheduling Updates

* **Immigration Data** - since each file contains data for a complete month I would opt for a monthly update as well. This is of course under the assumption that the exact date and location of the source data production is known.

* **Airport Data** - The data is updated daily, as the documentation states (https://datahub.io/core/airport-codes#automation). Since data quality proved rather poor here, with missing values and duplicates, I recommend a **daily update.**

* **Demographics Set** - The data on opendatasoft is from 2017 (last update). So my recommendation would be to **not update at all**. Since demographics hardly change this seems acceptable.
    * However since the United States Census Bureau has more datasets available it could also make sense to receive all historic data and then wait for **yearly updates.

#### Alternative Scenarios

**The data was increased by 100x.**

As explained above already only the airport data needs a daily update. The other sources change only on monthly and yearly basis.

_However_ airport data cannot increase by 100x since this would require an equal construction of airports, presumably. The same affects the demographics data which _is already aggregated_. So an increase makes no sense here.

Even if the size of this data increased I cannot see any performance issues coming up from the data model perspective.

But IF Immigration data would have a size of 50 GB per month instead of (roughly) 500 MB we would face some issues:
* Current processing time, including QA checks is 25 minutes
* In theory a linear increase of runtime would mean, the ETL script would have a processing time of 50 hours (estimated 30 mins for current size)
* This means (at minimum) a 100x **cost increase** to run it in AWS
    * So not only for cost reasons does it make sense then to select a different approach, but also because of possible scaling issues in Spark
* **Spark** tries to keep all data in memory so reading this data requires a large instance (e.g. `r4.4xlarge` which is memory-optimized)
    * **Recommendation 1: choose a larger instance**
* Another (probably more cost-effective way) is to split up the data (which also reqires compute power btw)
    * **Recommendation 2: check if data can be splitted, e.g. by US State or Airport and then imported in parallel**
    


**The data populates a dashboard that must be updated on a daily basis by 7am every day.**

I recommend to setup a daily data quality check and update of the data which runs during the night. Given the current runtime the update should start before 6:30 am.


**The database needed to be accessed by 100+ people.

Even if 100+ people needed access to the data I believe a large enough Redshift instance should be able to serve this requirement.

For less customers, even a small Postgres instance could be sufficient.

However if access should be granted it requires a "form of access" like a web gui - which needs to be taken into account when designing the overall system.